https://medium.com/dataseries/how-to-scrape-millions-of-tweets-using-snscrape-195ee3594721

## SENTIMENT ANALYSIS


In [ ]:
pip install textblob

In [ ]:
from textblob import TextBlob
#blob = TextBlob(df['Clean'][2])

In [ ]:
df['Sentiment_textblob'] = 0

In [ ]:
gme3['Sentiment_textblob'] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
gme3 = gme3.reset_index().drop(columns = ['index'])

In [ ]:
for i in range(gme3.shape[0]):
  blob = TextBlob(gme3['content'][i])
  gme3.iloc[i,2] = blob.sentiment.polarity

In [ ]:
pip install vaderSentiment

     |████████████████████████████████| 125 kB 2.0 MB/s 


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
gme3['Sentiment_vader'] = 0

In [ ]:
sid_obj = SentimentIntensityAnalyzer()

In [ ]:
for i in range(gme3.shape[0]):
  sentence = gme3['content'][i]
  sentiment_dict = sid_obj.polarity_scores(sentence)
  gme3.iloc[i,3] = sentiment_dict['compound']

In [ ]:
gme3

,date,content,Sentiment_textblob,Sentiment_vader
0,2021-09-18 23:56:45+00:00,confirms drs protect share phantom share put d...,-0.200000,0.7717
1,2021-09-18 23:53:47+00:00,aggressivevolatile high short squeeze stock po...,0.032000,-0.3182
2,2021-09-18 23:50:11+00:00,spacexs private inspiration4 mission splash sa...,0.250000,0.4939
3,2021-09-18 23:50:00+00:00,alert run gain daily alert one time fee beat t...,0.000000,0.7783
4,2021-09-18 23:48:38+00:00,stop,0.000000,-0.2960
...,...,...,...,...
43354,2021-08-19 00:02:08+00:00,almost half like good tweet date would funny b...,0.261111,0.7755
43355,2021-08-19 00:01:58+00:00,why fuck company like hold work person fund el...,-0.350000,-0.2500
43356,2021-08-19 00:00:38+00:00,stock crater that cost,0.000000,0.0000
43357,2021-08-19 00:00:31+00:00,wait coworkers moass specially boss make fun a...,0.319048,0.7845


Cleaning up - NLP 

In [ ]:
!pip install unidecode
!pip install word2number
!pip install contractions

import re
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy

import unidecode
from word2number import w2n
import contractions
import nltk

import string #punctuation removal

nltk.download('stopwords')
stop = stopwords.words('english')                      #English stopwords 
nlp = spacy.load('en', disable=['parser', 'ner'])

def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text

def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
    return text

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

for w in nlp.Defaults.stop_words:          ##Append stopwords from spacy into nltk's stopwords list 
  if w not in stop:
    stop.append(w)

stop.append("&amp")    #Common redundant word

def remove_user_ticker(text):
  return re.subn('[@$]\w+', '', text)[0].strip()    ## Remove all the @someone and $GMT whatever 

def remove_links(text):
  return re.sub(r'http\S+', '', text)         ## Remove all the twitter references https://t.co/somehting

def remove_symbols_punctuation(text):      ## Removing all the symbols and punctuations in text --> mainly # and ! , . 
  return text.translate(str.maketrans('', '', string.punctuation))

def remove_digits(text):   ## Remove any number present in text on its own --> likely $123 became 123 after removing symbols
    sen = []
    for w in text.split():
        if not w.isdigit():
            sen.append(w)
    return ' '.join(sen)

def spacy_lemmatizer(text):    ## Lemmatize the words back to original form instead of stemming
  sen = []
  for w in nlp(text):
    if w.lemma_ != "-PRON-":    ## spacy changes my etc to -PRON- (i.e pronoun) useless so remove
     sen.append(w.lemma_)
  return " ".join(sen)

def remove_single_char(text):   ## Empirically single & double character words (i, to, is) I think don't add much sentiment value so i removed
  sen = []
  for w in text.split():
    if len(w) > 2:
      sen.append(w)
  return " ".join(sen)

     |████████████████████████████████| 235 kB 28.5 MB/s 
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5580 sha256=cc0857499558528be280065411d7c3ebe1c34759248682e6ea115b014a0020e7
  Stored in directory: /root/.cache/pip/wheels/4b/c3/77/a5f48aeb0d3efb7cd5ad61cbd3da30bbf9ffc9662b07c9f879
Successfully built word2number
     |████████████████████████████████| 321 kB 30.6 MB/s 
     |████████████████████████████████| 284 kB 40.4 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85451 sha256=d7d4c7908da824058b655cd19e65d2274e089cfdbf3fe64185a88ff31110ca12
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def text_cleaner(df, content):
  df[content] = df[content].apply(lambda s: remove_accented_chars(s))
  df[content] = df[content].apply(lambda s: expand_contractions(s))
  df[content] = df[content].apply(lambda s: remove_emoji(s))
  df[content] = df[content].apply(lambda x: " ".join(x.lower() for x in x.split() if x not in stop))
  df[content] = df[content].apply(lambda s: remove_user_ticker(s))
  df[content] = df[content].apply(lambda s: remove_links(s))
  df[content] = df[content].apply(lambda s: remove_symbols_punctuation(s))
  df[content] = df[content].apply(lambda s: remove_digits(s))
  df[content] = df[content].apply(lambda s: spacy_lemmatizer(s))
  df[content] = df[content].apply(lambda s: remove_single_char(s))

In [ ]:
text_cleaner(amc2, 'content')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead